In [24]:
# Todos los candidatos de avisos posibles
avisos_diff = df_avisos_diff.idaviso
avisos_diff.head()

0       8725750
1      17903700
2    1000610287
3    1001135716
4    1001326344
Name: idaviso, dtype: int64

In [1]:
import pandas as pd
import numpy as np
import time, os
import random
from datetime import datetime
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
import lightgbm as lgb

## Init

In [2]:
df_test = pd.read_csv('./data/ejemplo_de_solucion.csv')
df_avisos = pd.read_csv('./data/avisos/avisos_filtrados.csv')
print(df_test.shape, df_avisos.shape)

(312464, 2) (18362, 7)


## Baseline_1: Azar

In [3]:
# Todos los candidatos de avisos posibles
avisos = df_avisos.idaviso
avisos.head()

0       8725750
1      17903700
2    1000610287
3    1001135716
4    1001326344
Name: idaviso, dtype: int64

In [22]:
#Postulantes a recomendar avisos
postulantes = df_test.idpostulante
postulantes.head()

0    ZaO5
1    NdJl
2    5kNq
3    Ez8J
4    erL9
Name: idpostulante, dtype: object

In [5]:
avisoRandom = random.choices(avisos)[0]
avisoRandom

1112315908

In [6]:
list_pred = []

for row in postulantes:
    row_pred = dict()
    row_pred["idaviso"] = random.choices(avisos)[0]
    row_pred["idpostulante"] = row
    list_pred.append(row_pred)
   

In [7]:
df_baseLine_1 = pd.DataFrame(list_pred)
df_baseLine_1.head()

,idaviso,idpostulante
0,1112300756,ZaO5
1,1112431016,NdJl
2,1112279042,5kNq
3,1112226644,Ez8J
4,1112207834,erL9


In [10]:
#df_baseLine_1.to_csv("./submissions/baseLine_1.csv", index=False)
# Resultado Kaggle: 0.00017

## Baseline_2: Predigo un aviso y lo repito en todas las apariciones

In [8]:
df_postulantes = pd.read_csv('./data/postulantes/postulantes_train_full.csv')
df_train = pd.read_csv('./data/postulaciones/postulaciones_train.csv')

In [19]:
#df_train = df_train[:10000]

In [10]:
df_postulantes

,idpostulante,fechanacimiento,sexo,nombre,estado
0,NM5M,1970-12-03,FEM,Secundario,Graduado
1,5awk,1962-12-04,FEM,Universitario,Graduado
2,ZaO5,1978-08-10,FEM,Terciario/Técnico,Graduado
3,NdJl,1969-05-09,MASC,Posgrado,En Curso
4,eo2p,1981-02-16,MASC,Secundario,Graduado
...,...,...,...,...,...
447904,GNZr6xw,1997-08-17,MASC,Universitario,En Curso
447905,OqrJ8zv,1998-06-23,FEM,Terciario/Técnico,Graduado
447906,X95VzpK,1998-02-25,MASC,Universitario,En Curso
447907,QNr9A2N,1992-08-01,MASC,Master,En Curso


In [11]:
df_train_full =  df_train.merge(df_postulantes, on='idpostulante', how='left')
df_train_full = df_train_full[['idaviso','idpostulante','fechanacimiento','sexo','nombre','estado']]
print(df_train_full.isna().sum())
df_train_full

idaviso                 0
idpostulante            0
fechanacimiento    247920
sexo               179938
nombre             179938
estado             179938
dtype: int64


,idaviso,idpostulante,fechanacimiento,sexo,nombre,estado
0,1112257047,NM5M,1970-12-03,FEM,Secundario,Graduado
1,1111920714,NM5M,1970-12-03,FEM,Secundario,Graduado
2,1112346945,NM5M,1970-12-03,FEM,Secundario,Graduado
3,1112345547,NM5M,1970-12-03,FEM,Secundario,Graduado
4,1112237522,5awk,1962-12-04,FEM,Universitario,Graduado
...,...,...,...,...,...,...
6468684,1112345900,5mPJNDG,1998-07-21,FEM,Universitario,En Curso
6468685,1112334789,5mPJNDG,1998-07-21,FEM,Universitario,En Curso
6468686,1112334784,5mPJNDG,1998-07-21,FEM,Universitario,En Curso
6468687,1112334806,5mPJNDG,1998-07-21,FEM,Universitario,En Curso


In [12]:
df_test_full =  df_test.merge(df_postulantes, on='idpostulante', how='left')
df_test_full = df_test_full[['idaviso','idpostulante','fechanacimiento','sexo','nombre','estado']]
print(df_test_full.isna().sum())
df_test_full = df_test_full[['idpostulante', 'fechanacimiento','sexo','nombre','estado']]
df_test_full

idaviso                0
idpostulante           0
fechanacimiento    26286
sexo               20790
nombre             20790
estado             20790
dtype: int64


,idpostulante,fechanacimiento,sexo,nombre,estado
0,ZaO5,1978-08-10,FEM,Terciario/Técnico,Graduado
1,NdJl,1969-05-09,MASC,Posgrado,En Curso
2,5kNq,1973-02-08,MASC,Otro,En Curso
3,Ez8J,1968-12-18,MASC,Universitario,Abandonado
4,erL9,1977-10-02,MASC,Terciario/Técnico,Graduado
...,...,...,...,...,...
312459,EzemRA2,1992-06-22,MASC,Master,Graduado
312460,Nzr1RRJ,2000-04-08,MASC,Secundario,Graduado
312461,QNrBRk5,1990-10-07,MASC,Secundario,En Curso
312462,96XvNjP,1972-03-21,FEM,Secundario,Graduado


In [13]:
df_train_full['idpostulante'] = df_train_full['idpostulante'].astype('category')
df_train_full['fechanacimiento'] = df_train_full['fechanacimiento'].astype('category')
df_train_full['sexo'] = df_train_full['sexo'].astype('category')
df_train_full['nombre'] = df_train_full['nombre'].astype('category')
df_train_full['estado'] = df_train_full['estado'].astype('category')

df_test_full['idpostulante'] = df_test_full['idpostulante'].astype('category')
df_test_full['fechanacimiento'] = df_test_full['fechanacimiento'].astype('category')
df_test_full['sexo'] = df_test_full['sexo'].astype('category')
df_test_full['nombre'] = df_test_full['nombre'].astype('category')
df_test_full['estado'] = df_test_full['estado'].astype('category')

In [14]:
full_x = df_train_full[['idpostulante', 'fechanacimiento','sexo','nombre','estado']]
full_y = df_train_full[['idaviso']]
X_train, X_test, y_train, y_test = train_test_split(full_x, full_y, test_size=0.3, random_state=0)

In [15]:
lgb_train = lgb.Dataset(full_x, full_y)

In [16]:
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
# acotar hojas y profundidad
# Staking

In [17]:
print('Starting training...')
# train
#gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval,early_stopping_rounds=5)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20)
# save model to file
#gbm.save_model('model.txt')

Starting training...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049311 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


In [18]:
print('Starting predicting...')
# predict
y_pred_lgb = gbm.predict(X_train, num_iteration=gbm.best_iteration)
# eval
#print('RMSE: %.4f' % np.sqrt(mean_squared_error(y_test, y_pred_lgb)))

Starting predicting...


In [35]:
t_0 = time.time()
y_pred = gbm.predict(df_test_full, num_iteration=gbm.best_iteration)
print ("tiempo: ", time.time() - t_0)

tiempo:  1.1920278072357178


In [21]:
len(y_pred)

312464

In [23]:
y_pred[8]

1111781447.6673856

In [25]:
# La lista de predicciones tiene valores que no existen en la lista de "avisos", con lo cual tengo que buscar para cada prediccion el valor mas cercano en dicha lista.

In [26]:
avisos_list = avisos.values.tolist()

In [28]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [29]:
find_nearest(avisos_list,1111781447 )

1111782047

In [30]:
y_pred_avisos = []
for index in tqdm_notebook(y_pred):
    y_pred_avisos.append(find_nearest(avisos_list,index.astype(int)))

In [36]:
def write_submit(df_test,prediccion,fileName):
    submit = pd.DataFrame({'idaviso':prediccion})
    submit['idaviso'] =  submit['idaviso'].astype(int)
    submit['idpostulante'] = df_test.idpostulante
    submit.to_csv("./submissions/"+fileName+".csv", index=False)

In [37]:
write_submit(df_test,y_pred_avisos,'baseline_2' )

In [33]:
1111782047 in avisos_list

True

In [34]:
for element in tqdm_notebook(y_pred_avisos):
    boolean = element in avisos_list
    if boolean == False:
        print(element, "no esta en la lista de avisos")
    

## Baseline 3
### Elimino los avisos con duracion menos duracion (25%).

In [66]:
df_avisos_diff = df_avisos

In [74]:
df_avisos_diff['online_hasta'] = pd.to_datetime(df_avisos_diff['online_hasta'])
df_avisos_diff['online_desde'] = pd.to_datetime(df_avisos_diff['online_desde'])

In [75]:
(df_avisos_diff.loc[1]['online_hasta'] - df_avisos_diff.loc[1]['online_desde']).days

28

In [76]:
df_avisos_diff['diff'] = (df_avisos_diff['online_hasta'] - df_avisos_diff['online_desde'])

In [77]:
df_avisos_diff.sort_values('diff', ascending=False).head()

,idaviso,titulo,nivel_laboral,nombre_area,denominacion_empresa,online_desde,online_hasta,diff
703,1112219332,Ingeniero Electrónico Jr,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Adecco -Región GBA SUR,2018-01-15,2018-04-17,92 days
2845,1112224292,Social Media Manager,Senior / Semi-Senior,Marketing,Wall Chase,2018-01-15,2018-04-17,92 days
10786,1112208757,Repositor externo Zona Norte,Senior / Semi-Senior,Almacén / Depósito / Expedición,Adecco -Sales & Marketing,2018-01-15,2018-04-17,92 days
3297,1112232024,Supervisor de Punto de Venta,Senior / Semi-Senior,Comercial,Hidalgo & Asociados,2018-01-15,2018-04-17,92 days
5950,1112247209,Analista de Recursos Humanos para Importante M...,Senior / Semi-Senior,Recursos Humanos,Focalizar Capital Humano,2018-01-15,2018-04-17,92 days


In [79]:
print(df_avisos_diff['diff'].describe())
print()
print(len(df_avisos_diff.loc[df_avisos_diff['diff'] > '11 days']))

count                      18362
mean     23 days 04:55:11.011872
std      16 days 03:56:31.733150
min              0 days 00:00:00
25%             11 days 00:00:00
50%             26 days 00:00:00
75%             29 days 00:00:00
max             92 days 00:00:00
Name: diff, dtype: object

13519


In [80]:
df_avisos_diff = df_avisos_diff.loc[df_avisos['diff'] > '11 days']
df_avisos_diff

,idaviso,titulo,nivel_laboral,nombre_area,denominacion_empresa,online_desde,online_hasta,diff
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Senior / Semi-Senior,Comercial,VENTOR,2018-01-15,2018-02-10,26 days
1,17903700,Enfermeras,Senior / Semi-Senior,Salud,Farmacias Central Oeste,2018-03-20,2018-04-17,28 days
2,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,2018-01-15,2018-03-17,61 days
3,1001135716,Vendedor Viajante TUCUMAN/SANTIAGO DEL ESTERO,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,2018-01-15,2018-04-16,91 days
4,1001326344,Vendedor Viajante RECONQUISTA/AVELLANEDA,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,2018-01-15,2018-03-27,71 days
...,...,...,...,...,...,...,...,...
18353,1112407368,Desarrollador Franquicias REMAX - Profesionale...,Senior / Semi-Senior,Marketing,GB Consultores - RRHH,2018-03-16,2018-04-13,28 days
18354,1112407434,Port Folio nuevos negocios,Senior / Semi-Senior,Desarrollo de Negocios,Importante Empresa,2018-03-16,2018-04-10,25 days
18356,1112407469,Asuntos Regulatorios,Senior / Semi-Senior,Farmacéutica,GI GROUP Argentina,2018-03-16,2018-04-05,20 days
18359,1112407609,Tecnico sector moldes,Senior / Semi-Senior,Laboratorio,GAES S A,2018-03-16,2018-04-13,28 days


In [81]:
# Todos los candidatos de avisos posibles
avisos_diff = df_avisos_diff.idaviso
avisos_diff = avisos_diff.tolist()

In [84]:
avisoRandom = random.choices(avisos_diff)[0]
avisoRandom 

1112289757

In [85]:
list_pred_3 = []

for row in postulantes:
    row_pred = dict()
    row_pred["idaviso"] = random.choices(avisos_diff)[0]
    row_pred["idpostulante"] = row
    list_pred_3.append(row_pred)
   

In [86]:
df_baseLine_3 = pd.DataFrame(list_pred_3)
df_baseLine_3.head()

,idaviso,idpostulante
0,1112169167,ZaO5
1,1112418975,NdJl
2,1112296292,5kNq
3,1112322919,Ez8J
4,1112424893,erL9


In [88]:
df_baseLine_3.to_csv("./submissions/baseLine_3.csv", index=False)
# Resultado Kaggle: 0.00015